In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
import time

In [2]:
train = pd.read_csv('D:/DSML/Semester 2/CS4248/Group Project/Project Code/dataset/raw_data/fulltrain.csv', names=['label', 'text'])
X_train = train['text']
y_train = train['label']
tfidf = TFIDF().fit(X_train)
X_train = tfidf.transform(X_train)

In [3]:
# define the model
# candidate_models: MultinomialNB, ComplementNB, BernoulliNB              # Train,Test
# model = MultinomialNB()                                                 # 0.592,0.319
# model = CalibratedClassifierCV(MultinomialNB(), cv=2, method='isotonic')# 0.791,0.550
# model = CalibratedClassifierCV(MultinomialNB(), cv=2, method='sigmoid') # 0.675,0.408
# model = ComplementNB()                                                  # 0.839,0.484
# model = CalibratedClassifierCV(ComplementNB(), cv=2, method='isotonic') # 0.858,0.580
# model = CalibratedClassifierCV(ComplementNB(), cv=2, method='sigmoid')  # 0.850.0.567
# model = BernoulliNB()                                                   # 0.778,0.596
# model = CalibratedClassifierCV(BernoulliNB(), cv=2, method='isotonic')  # 0.786,0.593
model = CalibratedClassifierCV(BernoulliNB(), cv=3, method='sigmoid')  # 0.781,0.610

model.fit(X_train, y_train)
y_pred = model.predict(X_train)

# Use f1-macro as the metric
score = f1_score(y_train, y_pred, average='macro')
print('Train F1 score = {}'.format(score))

Train F1 score = 0.7810799562161455


In [6]:
# test data for original dataset
test = pd.read_csv('D:/DSML/Semester 2/CS4248/Group Project/Project Code/dataset/raw_data/balancedtest.csv', names=['label', 'text'])
X_test = test['text']
y_test = test['label']
X_test = tfidf.transform(X_test)
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
print('Test F1 score = {}'.format(score))
print(time.time() - time0, 's')
TEXT_LABELS = {1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable'}
class_names = list(TEXT_LABELS.values())
print(classification_report(y_test, y_pred, target_names=class_names))
cm = confusion_matrix(y_test, y_pred)
print(cm)

Test F1 score = 0.6098074991771212
204.1195969581604 s
              precision    recall  f1-score   support

      Satire       0.61      0.75      0.68       750
        Hoax       0.50      0.34      0.41       750
  Propaganda       0.60      0.79      0.68       750
    Reliable       0.78      0.59      0.68       750

    accuracy                           0.62      3000
   macro avg       0.62      0.62      0.61      3000
weighted avg       0.62      0.62      0.61      3000

[[565 183   0   2]
 [164 258 290  38]
 [ 71   4 592  83]
 [119  74 111 446]]


In [5]:
# test data for covid dataset
test = pd.read_csv('D:/DSML/Semester 2/CS4248/Group Project/Project Code/covid_unreliable_news_cleaned.csv')
X_test = test['content_text']
y_test = test['class']
for i in range(len(y_test)):
    if y_test[i] == "Reliable News":
        y_test[i] = 4
    if y_test[i] == "Hoax":
        y_test[i] = 2
    if y_test[i] == "Propaganda":
        y_test[i] = 3
    if y_test[i] == "Satire":
        y_test[i] = 1
y_test = y_test.astype(np.int64)
X_test = tfidf.transform(X_test)
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
print('Test F1 score = {}'.format(score))
print(time.time() - time0, 's')
TEXT_LABELS = {1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable'}
class_names = list(TEXT_LABELS.values())
print(classification_report(y_test, y_pred, target_names=class_names))
cm = confusion_matrix(y_test, y_pred)
print(cm)

Test F1 score = 0.5122060008767559
92.30877041816711 s
              precision    recall  f1-score   support

      Satire       0.76      1.00      0.86        31
        Hoax       0.64      0.47      0.54        30
  Propaganda       0.50      0.76      0.60        29
    Reliable       0.08      0.03      0.05        30

    accuracy                           0.57       120
   macro avg       0.49      0.56      0.51       120
weighted avg       0.49      0.57      0.51       120

[[31  0  0  0]
 [ 1 14 10  5]
 [ 0  0 22  7]
 [ 9  8 12  1]]
